In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=256, shuffle=False, pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(in_features=32 * 8 * 8, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Function to train the model
def train_model(optimizer_type, lr, epochs=20):
    model = SimpleCNN().to(device)
    criterion = nn.CrossEntropyLoss()

    if optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    elif optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_type == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr)
    elif optimizer_type == 'AdamW':
        optimizer = optim.AdamW(model.parameters(), lr=lr)

    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

    print(f'Training with {optimizer_type} completed.')

    return model

# Function to evaluate the model
def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0

    # For calculating accuracy for individual classes
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Calculate accuracy for individual classes
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()  # Add correct predictions for this class
                class_total[label] += 1  # Increment total samples for this class

    # Overall accuracy of the model
    accuracy = 100 * correct / total

    # Accuracy for each class
    class_accuracies = []
    for i in range(10):
        if class_total[i] > 0:  # Prevent division by zero
            class_accuracies.append(100 * class_correct[i] / class_total[i])
        else:
            class_accuracies.append(0)

    return accuracy, class_accuracies

# Function to run evaluation multiple times and average the results
def run_multiple_evaluations(optimizer_type, lr, epochs=20, repetitions=3):
    total_accuracy = 0
    total_class_accuracies = np.zeros(10)

    for i in range(repetitions):
        print(f"\nRepetition {i + 1}/{repetitions} with {optimizer_type}")
        model = train_model(optimizer_type, lr, epochs)
        accuracy, class_accuracies = evaluate_model(model)
        total_accuracy += accuracy
        total_class_accuracies += np.array(class_accuracies)

    avg_accuracy = total_accuracy / repetitions
    avg_class_accuracies = total_class_accuracies / repetitions

    print("======================================================================")
    print(f"\nAverage Overall Accuracy with {optimizer_type}: {avg_accuracy:.2f}%")
    for i in range(10):
        print(f"Average Accuracy for {classes[i]}: {avg_class_accuracies[i]:.2f}%")
    print("======================================================================")

# Run the evaluation 10 times for each optimizer and average the results
print("Evaluating SGD:")
run_multiple_evaluations(optimizer_type='SGD', lr=0.001, epochs=20, repetitions=3)

print("\nEvaluating Adam:")
run_multiple_evaluations(optimizer_type='Adam', lr=0.001, epochs=20, repetitions=3)

print("\nEvaluating RMSprop:")
run_multiple_evaluations(optimizer_type='RMSprop', lr=0.001, epochs=20, repetitions=3)

print("\nEvaluating AdamW:")
run_multiple_evaluations(optimizer_type='AdamW', lr=0.001, epochs=20, repetitions=3)


Files already downloaded and verified
Files already downloaded and verified
Evaluating SGD:

Repetition 1/3 with SGD
Epoch [1/20], Loss: 2.2904
Epoch [2/20], Loss: 2.2235
Epoch [3/20], Loss: 2.0858
Epoch [4/20], Loss: 1.9860
Epoch [5/20], Loss: 1.8931
Epoch [6/20], Loss: 1.8003
Epoch [7/20], Loss: 1.7289
Epoch [8/20], Loss: 1.6704
Epoch [9/20], Loss: 1.6139
Epoch [10/20], Loss: 1.5593
Epoch [11/20], Loss: 1.5072
Epoch [12/20], Loss: 1.4666
Epoch [13/20], Loss: 1.4319
Epoch [14/20], Loss: 1.3973
Epoch [15/20], Loss: 1.3710
Epoch [16/20], Loss: 1.3476
Epoch [17/20], Loss: 1.3248
Epoch [18/20], Loss: 1.3012
Epoch [19/20], Loss: 1.2798
Epoch [20/20], Loss: 1.2633
Training with SGD completed.

Repetition 2/3 with SGD
Epoch [1/20], Loss: 2.2870
Epoch [2/20], Loss: 2.1991
Epoch [3/20], Loss: 2.0327
Epoch [4/20], Loss: 1.8874
Epoch [5/20], Loss: 1.7872
Epoch [6/20], Loss: 1.7238
Epoch [7/20], Loss: 1.6687
Epoch [8/20], Loss: 1.6187
Epoch [9/20], Loss: 1.5717
Epoch [10/20], Loss: 1.5254
Epoch [

In [3]:
class SimpleCNNWithDropout(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(in_features=32 * 8 * 8, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=10)

        # Dropout layers
        self.dropout = nn.Dropout(p=0.5)  # 50% of neurons will be dropped

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = self.dropout(F.relu(self.fc1(x)))  # Apply dropout before the final layer
        x = self.fc2(x)
        return x

def train_model_with_dropout(optimizer_type, lr, epochs=20):
    model = SimpleCNNWithDropout().to(device)
    criterion = nn.CrossEntropyLoss()

    if optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    elif optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_type == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr)
    elif optimizer_type == 'AdamW':
        optimizer = optim.AdamW(model.parameters(), lr=lr)

    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

    print(f'Training with {optimizer_type} and Dropout completed.')
    return model

# Function to run evaluation multiple times and average the results
def run_multiple_evaluations(optimizer_type, lr, epochs=20, repetitions=3):
    total_accuracy = 0
    total_class_accuracies = np.zeros(10)

    for i in range(repetitions):
        print(f"\nRepetition {i + 1}/{repetitions} with {optimizer_type}")
        model = train_model_with_dropout(optimizer_type, lr, epochs)
        accuracy, class_accuracies = evaluate_model(model)
        total_accuracy += accuracy
        total_class_accuracies += np.array(class_accuracies)

    avg_accuracy = total_accuracy / repetitions
    avg_class_accuracies = total_class_accuracies / repetitions

    print("======================================================================")
    print(f"\nAverage Overall Accuracy with {optimizer_type}: {avg_accuracy:.2f}%")
    for i in range(10):
        print(f"Average Accuracy for {classes[i]}: {avg_class_accuracies[i]:.2f}%")
    print("======================================================================")

# Run the evaluation 10 times for each optimizer and average the results
print("Evaluating SGD with Dropout:")
run_multiple_evaluations(optimizer_type='SGD', lr=0.001, epochs=20, repetitions=3)

print("\nEvaluating Adam with Dropout:")
run_multiple_evaluations(optimizer_type='Adam', lr=0.001, epochs=20, repetitions=3)

print("\nEvaluating RMSprop with Dropout:")
run_multiple_evaluations(optimizer_type='RMSprop', lr=0.001, epochs=20, repetitions=3)

print("\nEvaluating AdamW with Dropout:")
run_multiple_evaluations(optimizer_type='AdamW', lr=0.001, epochs=20, repetitions=3)


100%|██████████| 170498071/170498071 [00:03<00:00, 49234192.72it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Evaluating SGD with Dropout:

Repetition 1/3 with SGD
Epoch [1/20], Loss: 2.2976
Epoch [2/20], Loss: 2.2711
Epoch [3/20], Loss: 2.1857
Epoch [4/20], Loss: 2.0784
Epoch [5/20], Loss: 2.0040
Epoch [6/20], Loss: 1.9417
Epoch [7/20], Loss: 1.8890
Epoch [8/20], Loss: 1.8331
Epoch [9/20], Loss: 1.7783
Epoch [10/20], Loss: 1.7233
Epoch [11/20], Loss: 1.6747
Epoch [12/20], Loss: 1.6357
Epoch [13/20], Loss: 1.5996
Epoch [14/20], Loss: 1.5710
Epoch [15/20], Loss: 1.5488
Epoch [16/20], Loss: 1.5281
Epoch [17/20], Loss: 1.5115
Epoch [18/20], Loss: 1.4948
Epoch [19/20], Loss: 1.4749
Epoch [20/20], Loss: 1.4662
Training with SGD and Dropout completed.

Repetition 2/3 with SGD
Epoch [1/20], Loss: 2.2904
Epoch [2/20], Loss: 2.2281
Epoch [3/20], Loss: 2.1081
Epoch [4/20], Loss: 2.0039
Epoch [5/20], Loss: 1.9160
Epoch [6/20], Loss: 1.8519
Epoch [7/20], Loss: 1.8050
Epoch [8/20], Loss: 1.7643
Epoch [9/20], Loss: 1.72